In [ ]:
# download to local
dx download --lightweight -fr data

In [ ]:
#get list for files have not been download successfully
#df =  pd.read_csv("file_paths.txt", header=None)
result_file_path = os.path.join('/Users/xiaoqianxiao/UKB/data/derivatives', 'output.txt')
df =  pd.read_csv("file_paths.txt")
df['filename'] = df['filepath'].str.split(r'[/]', expand=True)[5]
directory = '/Users/xiaoqianxiao/UKB/data/derivatives/downloaded_UKB'
file_names = os.listdir(directory)
df['done_files'] = df['filename'].apply(lambda x: (x in file_names)).replace((1,0),('True','False'))
df.loc[df['done_files'] == False]['filepath'].to_csv('file_paths_remain.txt', sep='\t', index=False, header=False)
df.loc[df['done_files'] == False]['filename'].to_csv(result_file_path, sep='\t', index=False, header=False)

In [ ]:
#unzip files
sourceData_path='/Users/xiaoqianxiao/UKB/data/derivatives/downloaded_UKB2'
timeseries_path='/Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB'
cd $sourceData_path
for f in *.zip; do unzip "$f" -d "${timeseries_path}/${f%.zip}"; done

In [ ]:
#transform to BIDS format
import os
import re
import shutil
# Set the path to your directory containing the files
base_dir = "/Users/xiaoqianxiao/UKB/data/derivatives/unziped_UKB"
target_dir = "/Users/xiaoqianxiao/UKB/data/derivatives/timeseries"
# Function to rename files to BIDS format
#session: Instance(2=Imaging visit; 3=First repeat imaging visit)
def rename_to_bids(base_dir):
    # Loop through each subdirectory in the base directory
    for subject_dir in os.listdir(base_dir):
        subject_path = os.path.join(base_dir, subject_dir)

        if os.path.isdir(subject_path):
            # Extract the subject ID from the folder name
            subject_id = re.split(r'[_\s;]+', subject_dir)[0]
            session_id =re.split(r'[_\s;]+', subject_dir)[2]
        
            # Loop through the files in the subject directory
            for filename in os.listdir(subject_path):
                space_id = re.split(r'[.\s;]+', filename)[1]
                if filename.endswith('.csv.gz'):
                    #sub-subID_ses-mri_run-runID_task-{task_name}_space-space.csv.gz
                    task_name = "rest"  # Modify this if you have different tasks
                    new_filename = f"sub-{subject_id}_ses-{session_id}_task-{task_name}_space-{space_id}.csv.gz"
                    new_file_path = os.path.join(target_dir, new_filename)

                    # Full path of the old file
                    old_file_path = os.path.join(subject_path, filename)

                    # Rename the file
                    os.rename(old_file_path, new_file_path)
                    #shutil.copy(old_file_path, new_file_path)
                    #print(f"Renamed: {old_file_path} to {new_file_path}")

# Call the function
rename_to_bids(base_dir)